## Preface



In [9]:
!pip install -r requirements.txt

import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

# Uncomment for use in Jupyter Notebook
%matplotlib widget

# If widget doesn't work, try this (for static images)
#%matplotlib inline

## Consumer Demand for Food



-   Minimum cost model can be regarded as *prescriptive* or
    *descriptive*.  In the former case it&rsquo;s a way of saying what
    people *should* eat.  In the latter it&rsquo;s a way of saying what
    people *do* eat. 
    
    [FoodDemand in datahub](http://datahub.berkeley.edu/user-redirect/interact?account=ligon&repo=FoodDemand.git&branch=master&path)



### Questions



-   How do actual diets compare with minimum cost diets?
    -   How can changes in food prices affect demand for different kinds
        of food?
    -   How can changes in  income affect demand for different kinds of
        food?
    -   Is there a set of prices and incomes that ensure people will
        choose to consume a nutritious diet?



### Resources (food expenditures)



-   [ICRISAT Expenditure Data](https://docs.google.com/spreadsheets/d/13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo/)
    -   Philippine (Mindanao) Individual Food Consumption
    -   Indian NSS
    -   Malawi IHS (See appendix of [this IFPRI working paper](http://ebrary.ifpri.org/utils/getfile/collection/p15738coll2/id/128205/filename/128416.pdf))
    -   Indonesian Family Life Survey
    -   [http://surveys.worldbank.org/lsms](http://surveys.worldbank.org/lsms)
    -   US Nielsen or Consumer Expenditure Survey?



### Resources (Food conversion tables)



-   [https://ndb.nal.usda.gov/ndb/](https://ndb.nal.usda.gov/ndb/)
    -   FAO: [http://www.fao.org/fileadmin/templates/food_composition/documents/AnFooD2.0.xlsx](http://www.fao.org/fileadmin/templates/food_composition/documents/AnFooD2.0.xlsx)



### Consumer's problem



The basic consumer&rsquo;s problem considers the problem of allocating a
   budget $x$ to purchase consumption goods (including but not
   limited to different kinds of food).  The consumer is thought to
   try to choose goods to maximize their *utility*.  Utility in turn
   is a function that maps *consumption bundles* into the  real
   line&#x2014;one can think of the level of utility as corresponding to
   the consumer&rsquo;s happiness or satisfaction.

The problem is expressed mathematically as
$$
      \max_{c\in\R^n_{++}} U(c) \qquad\text{such that $p'c\leq x$,}
   $$
where $U:\R^n_{++}\rightarrow\R$ is generally assumed to be an
increasing, concave, and continuously differentiable function of
consumption; $c$ is the consumption bundle; $p$ is a vector of
prices for the various goods in $c$; and $x$ is the consumer&rsquo;s
total budget.



#### Cobb-Douglas Example Utility Function



Let&rsquo;s consider a particular utility function which  takes the
    form
    $$
       U(c) = \sum_{i=1}^n\alpha_i\log c_i.
    $$
    This logarithmic form is sometimes called a *Cobb-Douglas*
    utility function, named after a mathematician Charles Cobb and an
    economist (later Senator) Paul Douglas, who devised the
    mathematical form as a way of modeling [production](https://en.wikipedia.org/wiki/Cobb%25E2%2580%2593Douglas_production_function).



In [10]:
from cfe import demands

# Cobb-Douglas utility is a special case of a more general class
def cobb_douglas_utility(alpha):
    """
    Define Cobb-Douglas utility function.
    """
    n = len(alpha)
    U = lambda c: demands.utility(c,alpha,[1]*n,[0]*n)

    return U

# Two good case:
alpha=[0.3,0.7] # Preference parameters

U = cobb_douglas_utility(alpha)

Now construct a 3-d plot of the utility function for us to visualize:



In [11]:
import plotly.offline as py
import plotly.graph_objs as go
#from mpl_toolkits import mplot3d
import numpy as np
#import matplotlib.pyplot as plt

#fig = plt.figure()
#ax = plt.axes(projection='3d')

# Commodity space
c1 = np.linspace(0.1, 10, 30)
c2 = np.linspace(0.1, 10, 30)

#C1,C2 = np.meshgrid(c1,c2)

Utils = np.array([[U([c_1,c_2]) for c_1 in c1] for c_2 in c2])

utils = [
    go.Surface(
        x=c1,
        y=c2,
        z=Utils,
        contours=go.surface.Contours(
            z=go.surface.contours.Z(
              show=True,
              usecolormap=True,
              highlightcolor="#42f462",
              project=dict(z=True)
            )
        )
    )
]

layout = go.Layout(
    title='Cobb-Douglas Utility',
    scene={'xaxis':{'title':'$c_1$'},
           'yaxis':{'title':'$c_2$'},
           'zaxis':{'title':'Utils'}})

fig = go.Figure(data=utils, layout=layout)

py.iplot(fig, filename='latex')

##### Questions



Some questions covering basic consumer demand that should be
     familiar.  If not, consult a textbook on intermediate
     microeconomics (such as [Perloff&rsquo;s textbook with calculus](https://www.amazon.com/Microeconomics-Applications-Calculus-Pearson-Economics/dp/0134167384)).  Or
     for a concise high-level treatment see the excellent (and free!)
     [Introduction by Arne Hallam](http://www2.econ.iastate.edu/faculty/hallam/Microeconomics/Intro-Micro.pdf).

1.  Economists would call the $(c_1,c_2)$ plane the &ldquo;commodity
    space&rdquo;.  Explain.
2.  The contour lines in the figure are called &ldquo;Indifference
    curves&rdquo;.  Why is this sensible?
3.  Economists call the partial derivatives of the utility function with respect to
    $c_1$ and $c_2$ *marginal utilities*.  Give a graphical
    interpretation of the marginal utility of $c_1$ (holding
    $c_2$ fixed).
4.  Pick out a point on an indifference curve.  What is its slope
    in the $(c_1,c_2)$ plane?
5.  Economists call (minus) the slope in the previous question
    the consumer&rsquo;s *marginal rate of substitution*.  Explain.



##### The Budget Constraint



Now, how about  the budget constraint?



In [12]:
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import pandas as pd
from cfe import demands

py.init_notebook_mode(connected=True)

########### Play with these parameters
alpha=[0.3,0.7] # Preference parameters
beta=[1,1] # Curvature parameters

####################

U = lambda c: demands.utility(c,alpha,beta,0)

# Prices of goods
p = (1,2)

# Budget
x=2

c1 = np.linspace(0.1, x/p[0]*1.2, 30)
c2 = np.linspace(0.1, x/p[1]*1.2, 30)

Utils = np.array([[U([c_1,c_2]) for c_1 in c1] for c_2 in c2])

Ustar =  demands.marshallian.indirect_utility(x,p,alpha,1,0)

xs = [0,x/p[0],0,x/p[0]]
ys = [x/p[1],0,x/p[1],0]
zs = [Utils.min(),Utils.min(),Utils.max()*1.1,Utils.max()*1.1]

budget= [{
    'type': 'mesh3d',        
    'x': xs,
    'y': ys,
    'z': zs,
    'delaunayaxis':'x',
    'color': 'black',
    'opacity': 1.0,
}]

utils = [
    go.Surface(
        x=c1,
        y=c2,
        z=Utils,
        contours=go.surface.Contours(
            z=go.surface.contours.Z(
              show=True,
              usecolormap=True,
              highlightcolor="#42f462",
              project=dict(z=True)
            )
        )
    )
]

data = budget + utils


layout = go.Layout(
    title='Cobb-Douglas Utility',
    scene={'xaxis':{'title':'$\\alpha$'},
           'yaxis':{'title':'$c_2$'},
           'zaxis':{'title':'Utils'}})

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='latex')

The budget constraint here is the black line/form.



###### Questions



1.  How is the budget constraint determined by prices $p$ and budget $x$?
    1.  The figure above shows a tangency between the budget line and a
        particular indifference curve.  Show that this point &ldquo;solves&rdquo;
        the consumer&rsquo;s problem, in the sense that it maximizes utility
        subject to the consumer&rsquo;s budget constraint.



##### Demands



We&rsquo;ve seen how one can find demands given a utility function and a
budget constraint.  As one varies the price of a particular good
(given a fixed budget and holding other prices fixed) one traces out
the *demand curve* for the good.

The following code does this for the same utility function we explored
above.  Note that for obscure historical reasons ([Alfred](https://en.wikipedia.org/wiki/Alfred_Marshall) & [Mary Paley
Marshall](https://en.wikipedia.org/wiki/Mary_Paley_Marshall) are to blame) it&rsquo;s conventional to plot the independent
variable &ldquo;price&rdquo; on the vertical axis, instead of the horizontal.



In [13]:
from cfe.demands import marshallian
import numpy as np
import matplotlib.pyplot as plt

alpha=[0.3,0.7] # Preference parameters

# Demand for c_1 as a function of own price
D1 = lambda p1,p2=1,x=1: marshallian.demands(x,(p1,p2),alpha,1,0)[0]

P = np.linspace(.5,5,20)

plt.plot([D1(p1) for p1 in P],P)
plt.xlabel('$c_1$')
plt.ylabel('$p_1$')

plt.show()

FigureCanvasNbAgg()

###### Questions



1.  How does the demand curve for $c_1$ change if the price of
    the other good changes?
    1.  How does the demand curve for $c_1$ change if the budget $x$
        changes?



##### Engel Curves



The &ldquo;demand curve&rdquo; traces out how demand for a good changes as
     its price changes.  If prices remain fixed and the consumer&rsquo;s
     *budget* increases this describes what we call an [Engel](https://en.wikipedia.org/wiki/Ernst_Engel) curve.



In [15]:
from cfe.demands import marshallian
import numpy as np
import matplotlib.pyplot as plt

plt.figure(2)

alpha=[0.3,0.7] # Preference parameters

# Demand for c_1 as a function of own price
E1 = lambda x,p1=1,p2=1: marshallian.demands(x,(p1,p2),alpha,1,0)

X = np.linspace(.1,10,20)

plt.plot(X,[E1(x)[0] for x in X],X,[E1(x)[1] for x in X])
plt.xlabel('$x$')
plt.ylabel('Consumptions')
plt.legend(('$c_1$','$c_2$'))
plt.title('Engel Curves')

plt.show()

FigureCanvasNbAgg()

###### Questions



1.  How do the Engel curves depend on prices?
    1.  In the Cobb-Douglas case the parameters $\alpha_i$ are
        sometimes called &ldquo;budget shares.&rdquo;  Why does this makes
        sense?



#### Constant Frisch Elasticity (CFE) Example Utility Function



The Cobb-Douglas utility function is a special case of a more
    general class of utility functions.  These allow different
    curvatures in the utility derived from each good, unlike the
    Cobb-Douglas case.  They take the form
    $$
       U(c) =
    \sum_{i=1}^n\alpha_i\frac{\beta_i}{\beta_i-1}(c_i^{1-1/\beta_i} -1).
    $$
    Thus, where the Cobb-Douglas case had an $n$-vector of
    parameters $\alpha$, the CFE case has two $n$-vectors, &alpha;
    and &beta;.



In [17]:
from cfe import demands

########### Play with these parameters
alpha=[0.3,0.7] # Preference parameters
beta=[5,1] # Curvature parameters

####################

U = lambda c: demands.utility(c,alpha,beta,0)

Now construct a 3-d plot of the utility function for us to visualize:



In [18]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.axes(projection='3d')

# Commodity space
c1 = np.linspace(0.1, 10, 30)
c2 = np.linspace(0.1, 10, 30)

C1,C2 = np.meshgrid(c1,c2)

Utils = np.array([[U([c_1,c_2]) for c_1 in c1] for c_2 in c2])

ax.plot_surface(C1, C2, Utils, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none',alpha=0.8)

ax.set_xlabel('$c_1$')
ax.set_ylabel('$c_2$')
ax.set_zlabel('Utility')

# Plot indifference curves
ax.contour(C1,C2,Utils,offset=Utils.min())
ax.contour(C1,C2,Utils, colors="k", linestyles="solid")

plt.show()

FigureCanvasNbAgg()

##### Demands



We&rsquo;ve seen how one can find demands given a utility function and a
budget constraint.  As one varies the price of a particular good
(given a fixed budget and holding other prices fixed) one traces out
the *demand curve* for the good.

The following code does this for the same utility function we explored
above.  Note that for obscure historical reasons it&rsquo;s conventional to
plot the independent variable &ldquo;price&rdquo; on the vertical axis, instead of
the  horizontal.



In [19]:
from cfe.demands import marshallian
import numpy as np
import matplotlib.pyplot as plt

# Demand for c_1 as a function of own price
D1 = lambda p1,p2=1,x=1: marshallian.demands(x,(p1,p2),alpha,beta,0)[0]

P = np.linspace(.5,5,20)

fig,ax = plt.subplots()


#### Play with  p2 & x below!
ax.plot([D1(p1,p2=1,x=1) for p1 in P],P)

ax.set_xlabel('$c_1$')
ax.set_ylabel('$p_1$')

plt.show()

FigureCanvasNbAgg()

###### Questions



1.  How does the demand curve for $c_1$ change if the price of
    the other good changes?
    1.  How does the demand curve for $c_1$ change if the budget $x$
        changes?



##### Engel Curves



The &ldquo;demand curve&rdquo; traces out how demand for a good changes as
     its price changes.  If prices remain fixed and the consumer&rsquo;s
     *budget* increases this describes what we call an [Engel](https://en.wikipedia.org/wiki/Ernst_Engel) curve.



In [20]:
from cfe.demands import marshallian
import numpy as np
import matplotlib.pyplot as plt

# Demand for c_1 as a function of own price
E1 = lambda x,p1=1,p2=1: marshallian.demands(x,(p1,p2),alpha,beta,0)

X = np.linspace(.1,10,20)

fig,ax = plt.subplots()

#### Play with prices p1 & p2 below!
Ecurves = list(zip(*[E1(x,p1=1,p2=1) for x in X]))

ax.plot(X,Ecurves[0],X,Ecurves[1])

ax.set_xlabel('$x$')
ax.set_ylabel('Consumptions')
ax.legend(('$c_1$','$c_2$'))
ax.set_title('Engel Curves')

plt.show()

FigureCanvasNbAgg()

###### Questions



1.  How do the Engel curves depend on prices?

